In [2]:
#Celda para verificar que el NETLOGO_HOME esta definido bien y cargar el NetLogo(sin GUI)
import os
from pynetlogo import NetLogoLink


netlogo_home = os.environ.get("NETLOGO_HOME")
if not netlogo_home:
    raise EnvironmentError(
        "No se ha definido la variable NETLOGO_HOME.\n"
        "Por favor, ejecuta: export NETLOGO_HOME=/ruta/a/NetLogo_6.3.0"
    )


netlogo = NetLogoLink(gui=False, netlogo_home=netlogo_home)



In [3]:

import os
import csv

def run_netlogo_simulation(model_path, output_csv_path, netlogo, ticks=35000):
    """
    Ejecuta la simulación NetLogo alternando entre escenarios en bloques:
      - 40 ticks “estabilidad”
      - 40 ticks “inflacion”
      - 40 ticks “recesion”
      - 25 ticks “escasez”
    y repite este ciclo hasta alcanzar al menos `ticks` ticks totales.
    Los datos de cada tick se vuelcan en el CSV: escenario, demanda_preajuste, inventario_preajuste,
    produccion_preajuste, precio_preajuste, volatilidad_precio, subida_brusca_precio, escasez
    """

    # comprobar existencia del .nlogo y carga del modelo con setup inicial
    if not os.path.isfile(model_path):
        raise FileNotFoundError(f"El modelo NetLogo no se encuentra en: {model_path}")
    netlogo.load_model(model_path)
    netlogo.command('setup')

    # configuracion del csv y de encabezados
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = [
            "demanda_preajuste",
            "inventario_preajuste",
            "produccion_preajuste",
            "precio_preajuste",
            "volatilidad_precio",
            "subida_brusca_precio",
            "escasez"
        ]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        ticks_count = 0
        escenarios = ["estabilidad", "inflacion", "recesion", "escasez"]

        # Repetir ciclos hasta alcanzar el número deseado de ticks
        while ticks_count < ticks:
            for e in range(4):
                escenario = escenarios[e]
                if escenario == "escasez":
                    escenario_ticks = 25
                else:
                    escenario_ticks = 40

                for _ in range(escenario_ticks):
                    if ticks_count >= ticks:
                        break  

                   
                    run_tick_on_escenario(escenario, netlogo, writer)
                    ticks_count += 1

                if ticks_count >= ticks:
                    break

            #  cada ciclo de 40 + 40 + 40 + 25 para los diferentes escenarios siendo escasez el de 25 se vuelve al estado inicial con el setup
            if ticks_count < ticks:
                a = escenarios[2]
                b = escenarios[3]
                escenarios[3] = a
                escenarios[2] = b
                netlogo.command('setup')

    # Cerrar workspace de NetLogo
    netlogo.kill_workspace()
    print(f"Simulación completada: {ticks_count} ticks. Datos escritos en '{output_csv_path}'.")


def run_tick_on_escenario(escenario, netlogo, writer):
    """
    Cambia a 'escenario', ejecuta un paso 'go', lee las variables globales y escribe en el CSV.
    """
    netlogo.command(f'set escenario "{escenario}"')
    netlogo.command('go')

    demanda_preajuste    = netlogo.report('demanda_preajuste')
    inventario_preajuste = netlogo.report('inventario_preajuste')
    produccion_preajuste = netlogo.report('produccion_preajuste')
    precio_preajuste     = netlogo.report('precio_preajuste')
    volatilidad_precio   = netlogo.report('volatilidad_precio')
    subida_brusca        = netlogo.report('subida_brusca_precio?')
    escasez_flag         = netlogo.report('escasez?')

    writer.writerow({
        "demanda_preajuste":     demanda_preajuste,
        "inventario_preajuste":  inventario_preajuste,
        "produccion_preajuste":  produccion_preajuste,
        "precio_preajuste":      precio_preajuste,
        "volatilidad_precio":    round(volatilidad_precio, 5),
        "subida_brusca_precio":  int(subida_brusca),
        "escasez":               int(escasez_flag)
    })




In [ ]:

nLogoFilePath = "simulacionDefinitiva.nlogo"
exportFilePath = "simulacionEconomica.csv"

run_netlogo_simulation(nLogoFilePath, exportFilePath, netlogo)

KeyboardInterrupt: 